In [1]:
# Import required packages
from os import path 
import time
from datetime import datetime
import numpy as np
import pandas as pd
import torch
from torch import nn, Tensor
from torch.optim import AdamW
from tqdm import tqdm, trange
import zuko
from zuko.flows import Distribution, NSF
from zuko.distributions import DiagNormal, BoxUniform, Minimum
from zuko.flows import DistributionModule, FlowModule, Unconditional
from hnne import HNNE

from utils.settings import config
from utils.utils import *
from utils.model import *
from utils.robot import Robot

In [2]:
panda = Robot(verbose=False)
# data generation
J, P = data_collection(robot=panda, N=config.num_train_size)
# build dimension reduction model
F = posture_feature_extraction(J)
# hnne = get_hnne_model(J, P)
# get loader
loader = get_train_loader(J, P, F)
# loader = get_loader(J, P, hnne=hnne)
# get val loader
J, P = data_collection(robot=panda, N=config.num_val_size)
val_loader = get_test_loader(P, F)
# Build Generative model, NSF
# Neural spline flow (NSF) with 3 sample features and 5 context features
flow, optimizer, scheduler = get_flow_model()

hnne load successfully from /home/luca/ikpflow/weights/panda/hnne.pickle
Model load successfully from /home/luca/ikpflow/weights/panda/nsf.pth


In [3]:
Jb, Cb = next(iter(val_loader))
Jb.shape, Cb.shape

(torch.Size([128, 7]), torch.Size([128, 7]))

In [4]:
mu = torch.zeros(size=(7,))
NUM_DATA = 100
NUM_SAMPLES = 1000

In [5]:
df, err = test_l2_err(robot=panda, 
                      loader=val_loader, 
                      model=flow, 
                      num_data=NUM_DATA, 
                      num_samples=NUM_SAMPLES)
df.describe()

,l2_err,time_diff
count,100000.000000,100000.00000
mean,0.012720,0.08240
std,0.045122,0.02196
min,0.000009,0.08000
25%,0.002278,0.08000
50%,0.003694,0.08000
75%,0.006217,0.08000
max,1.324739,0.30000


In [6]:
iflow = get_iflow_model(flow, init_sample=mu, shrink_ratio=0.31)
df, err = test_l2_err(robot=panda, 
                      loader=val_loader, 
                      model=iflow, 
                      num_data=NUM_DATA, 
                      num_samples=NUM_SAMPLES)
df.describe()

,l2_err,time_diff
count,100000.000000,100000.00000
mean,0.008768,0.08240
std,0.036756,0.02196
min,0.000040,0.08000
25%,0.001438,0.08000
50%,0.002349,0.08000
75%,0.003854,0.08000
max,1.294425,0.30000


In [6]:
nflow = get_nflow_model(flow=flow)
df, err = test_l2_err(robot=panda, 
                      loader=val_loader, 
                      model=nflow, 
                      num_data=NUM_DATA, 
                      num_samples=NUM_SAMPLES)
df.describe()

,l2_err,time_diff
count,100000.000000,100000.0000
mean,0.003576,0.0802
std,0.011440,0.0014
min,0.000038,0.0800
25%,0.001303,0.0800
50%,0.002094,0.0800
75%,0.003237,0.0800
max,0.586376,0.0900


In [ ]:
from ikflow.utils import set_seed
from ikflow.model_loading import get_ik_solver

In [24]:
file_names = ['ang_errs_avg', 'ang_errs_min', 'F_avg', 'F_min', 'ikflow_ang', 'ikflow_l2', 'l2_errs_avg', 'l2_errs_min', 'n_evals']
exp_5_fig_dir = config.traj_dir + f'figs/exp_5_{datetime.now().strftime("%m%d%H%M")}/'
if not os.path.exists(path=exp_5_fig_dir):
    os.makedirs(exp_5_fig_dir)
set_seed()
num_trails = 100
num_generation = 100
num_ikflow_trails = num_generation
num_solutions = 500
# Build IKFlowSolver and set weights
ik_solver, hyper_parameters = get_ik_solver("panda__full__lp191_5.25m")
robot = ik_solver.robot
panda = Robot(verbose=False)

set_seed() - random int:  44
WorldModel::LoadRobot: /tmp/panda_arm_hand_formatted_link_filepaths_absolute.urdf
ndim_tot=7
dim_cond=8
joint mimic: no multiplier, using default value of 1 
joint mimic: no offset, using default value of 0 
URDFParser: Link size: 17
URDFParser: Joint size: 12
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jkinpylib/urdfs/panda/meshes/collision/link0.stl (595 verts, 200 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jkinpylib/urdfs/panda/meshes/collision/link1.stl (887 verts, 300 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jkinpylib/urdfs/panda/meshes/collision/link2.stl (889 verts, 300 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jkinpylib/urdfs/panda/meshes/collision/link3.stl (900 verts, 300 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jkinpylib/urdfs/panda/meshes/collision/link4.stl (9

In [19]:
def get_target_poses(loader):
    joint, ee = next(iter(loader))
    ee = ee.cpu().numpy()
    quaternions = np.zeros((len(ee), 4))
    quaternions[:, 0] = np.random.randn() * 2e-2 + 1
    target_poses = np.column_stack((ee[:, :3], quaternions))
    return target_poses

In [29]:
target_poses = get_target_poses(val_loader)
target_poses.shape

(128, 7)

In [34]:
# -> unrefined solutions
l2_errs = np.zeros((len(target_poses), NUM_SAMPLES))
time_diffs = np.zeros((len(target_poses), NUM_SAMPLES))

for i, target_pose in enumerate(target_poses):
    iksols, l2_err, _, _, _, dt = ik_solver.solve(
        target_pose, 
        n=NUM_SAMPLES,
        refine_solutions=False, 
        return_detailed=True)

    iksols = solutions.detach().cpu().numpy()
    dt = np.zeros_like(l2_err) + dt
    
    l2_errs[i] = l2_err
    time_diffs[i] = dt

l2_errs = l2_errs.reshape((-1))
time_diffs = time_diffs.reshape((-1))

    
df = pd.DataFrame(
    data=np.column_stack((l2_errs, time_diffs)),
    columns=["l2_err", "time_diff"],
)

Heads up: It may be faster to run solution_pose_errors() with pytorch directly on the cpu/gpu
Heads up: It may be faster to run solution_pose_errors() with pytorch directly on the cpu/gpu
Heads up: It may be faster to run solution_pose_errors() with pytorch directly on the cpu/gpu
Heads up: It may be faster to run solution_pose_errors() with pytorch directly on the cpu/gpu
Heads up: It may be faster to run solution_pose_errors() with pytorch directly on the cpu/gpu
Heads up: It may be faster to run solution_pose_errors() with pytorch directly on the cpu/gpu
Heads up: It may be faster to run solution_pose_errors() with pytorch directly on the cpu/gpu
Heads up: It may be faster to run solution_pose_errors() with pytorch directly on the cpu/gpu
Heads up: It may be faster to run solution_pose_errors() with pytorch directly on the cpu/gpu
Heads up: It may be faster to run solution_pose_errors() with pytorch directly on the cpu/gpu
Heads up: It may be faster to run solution_pose_errors() wit

In [42]:
df.describe()

,l2_err,time_diff
count,128000.000000,128000.000000
mean,0.090654,0.201037
std,0.227391,0.021056
min,0.000068,0.162164
25%,0.003930,0.191162
50%,0.006380,0.201805
75%,0.052177,0.208315
max,1.682320,0.343473
